# Android Security Bulletin Data Download

This notebook downloads data on CVEs reported on the monthly Android Security Bulletin, and merges it with the data provided by the CVE Program and with the data found on the AOSP git repo. The resulting aggregated data contains information about the dates of events of interest (e.g. commit to the repo, publication on the bulletin) for every patch published on the bulletin.

In [ ]:
import requests
import re
import dateutil
import json
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import pandas as pd
from os import path, listdir


BASE_URL = "https://source.android.com"
LIST_URL = "https://source.android.com/security/bulletin"
CVE_REGEX = re.compile("CVE-\d+-\d+")

In [ ]:
from dateutil import parser
def get_url_and_date(url_page):
  ret = []
  page = requests.get(url_page)
  doc = BeautifulSoup(page.content, "html.parser")
  rows = doc.find_all("tr")
  for row in rows[1:]:
    url = BASE_URL + row.find("td").find("a")["href"]
    date = parser.parse(row.find_all("td")[2].get_text().strip())
    ret.append((url, date))
  return ret

urls = get_url_and_date(LIST_URL)

# filter urls
urls = [ (u, d) for u, d in urls if path.basename(u) < "2023" and path.basename(u) >= "2017-06-01"]
urls

In [ ]:
! wget https://cve.mitre.org/data/downloads/allitems-cvrf.xml

In [ ]:
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2012.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2013.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2014.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2015.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2016.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2017.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2018.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2019.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2020.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2021.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2022.json.gz
! gzip -dkf nvdcve-1.1-2022.json.gz
! gzip -dkf nvdcve-1.1-2021.json.gz
! gzip -dkf nvdcve-1.1-2020.json.gz
! gzip -dkf nvdcve-1.1-2019.json.gz
! gzip -dkf nvdcve-1.1-2018.json.gz
! gzip -dkf nvdcve-1.1-2017.json.gz
! gzip -dkf nvdcve-1.1-2016.json.gz
! gzip -dkf nvdcve-1.1-2015.json.gz
! gzip -dkf nvdcve-1.1-2014.json.gz
! gzip -dkf nvdcve-1.1-2013.json.gz
! gzip -dkf nvdcve-1.1-2012.json.gz

In [ ]:
ns = {
    "vuln": "http://www.icasi.org/CVRF/schema/vuln/1.1",
    "cvrf": "http://www.icasi.org/CVRF/schema/cvrf/1.1",
}

def load_cvrf(filename):
  tree = ET.parse(filename)
  doc = tree.getroot()
  vulns = doc.findall("vuln:Vulnerability", namespaces=ns)
  ret = []
  for vuln in vulns:
    name = vuln.find("vuln:Title", namespaces=ns).text
    published = vuln.find("./vuln:Notes/vuln:Note[@Title='Published']", namespaces=ns)
    if published is not None:
      published = published.text
    modified = vuln.find("./vuln:Notes/vuln:Note[@Title='Modified']", namespaces=ns)
    if modified is not None:
      modified = modified.text
    ret.append({"cve": name, "published": published, "modified": modified})
  return pd.DataFrame(ret)

def get_modified(df, id):
  return df[df["cve"] == id].iloc[0].at["modified"]

def get_published(df, id):
  return df[df["cve"] == id].iloc[0].at["published"]

In [ ]:
# TODO: use this to filter Acknoledgements table?
def filter_table(table):
  cur = table.previous_sibling
  while cur.name != "h2":
    cur = cur.previous_sibling
  return cur.get_text().strip().lower() == "acknowledgements"

In [ ]:
def print_rows(rows):
  for row in rows:
    print(row.get_text())

def filter_rows(rows):
  return [row for row in rows if CVE_REGEX.search(row.get_text()) is not None]

def get_cves(info, get_commits=True):
  url, bulletin_date = info
  cves = []
  patch_level = path.basename(url)
  page = requests.get(url)
  doc = BeautifulSoup(page.content, "html.parser")
  tables = doc.find_all("table")
  #tables = [t for t in tables if not filter_table(t)]
  for table in tables:
    category = table.find_previous("h3").get_text().strip().lower()
    rows = filter_rows(table.find_all("td"))
    rows2 = table.find_all("td", string=CVE_REGEX)
    for row in rows:
      cells = row.parent.find_all("td")
      if category.startswith("google play"):
        cve = cells[1].get_text().strip()
        print(cve)
      else:
        cve = cells[0].get_text().strip()
      refs = []
      if get_commits:
        links = cells[1].find_all("a")
        if links is not None and CVE_REGEX.search(cve) is not None:
          for a in links:
            try:
              link = a["href"]
              (created, committed) = get_commit_date(link)
              ref = {"link": link, "created": created, "committed": committed}
              refs.append(ref)
            except:
              pass
      cur_cves = cve.split(',')
      for c in cur_cves:
        cves.append({"cve": c.strip(), "refs": refs, "bulletin_date": bulletin_date, "category": category, "patch_level": patch_level })
  return cves

def get_date_from_row(doc, head):
  try:
    row = doc.find("th", string=head).parent.find_all("td")
    return dateutil.parser.parse(row[1].text)
  except:
    return None

# matches codeaurora urls, making a distiction between cgit urls and gitweb urls.
# in case an url is a cgit url (i.e. group 1 matches), group 2 contains the path
# of the repo and group 3 contains the commit id.
# In case an url is a gitweb url, group 4 contains the path (including a '?p=...'
# parameter that shouuld be removed) and group 5 contains the commit id
CODEAURORA_RE = re.compile(r"https:\/\/(source\.codeaurora\.org|us\.codeaurora\.org\/cgit)\/quic\/(.*)\/commit\/?\?.*id=(.*)|https:\/\/www\.codeaurora\.org\/gitweb\/quic\/(.*\/?p=.*)\.git;a=commit;h=(.*)")

def fix_codeaurora_url(url):
  matches = CODEAURORA_RE.match(url)
  if matches:
    if matches.group(1):
      path = matches.group(2)
      commit = matches.group(3)
    else:
      path = matches.group(4).replace('?p=', '')
      commit = matches.group(5)
    url = f"https://git.codelinaro.org/clo/{path}/-/commit/{commit}"
  return url

# github web pages do not have all the information we can extract from cgit, so
# we can use the url for the old repo
def fix_github_url(url):
  return url.replace("github.com/torvalds/linux/commit/", "git.kernel.org/cgit/linux/kernel/git/torvalds/linux.git/commit/?id=")

def fix_url(url):
  url = fix_codeaurora_url(url)
  url = fix_github_url(url)
  return url

def get_codelinaro_date(doc):
  try:
    times = doc.findAll("time")
    return (dateutil.parser.parse(times[0]["datetime"]), dateutil.parser.parse(times[1]["datetime"]))
  except:
    return (None, None)

def get_commit_date(url):
  url = fix_url(url)
  print(f"\tget commit date from {url}...")
  page = requests.get(url)
  doc = BeautifulSoup(page.content, "html.parser")
  if "git.codelinaro.org" in url:
    return get_codelinaro_date(doc)
  return (get_date_from_row(doc, "author"), get_date_from_row(doc, "committer"))


In [ ]:
cves = []
for url in set(urls):
  print(f"download {url}...")
  cves.extend(get_cves(url))

In [ ]:
import numpy as np

df = pd.DataFrame(data=cves)
df_cat = df[df["bulletin_date"] >= "2017-06-01"]
df_cat.groupby("category")["bulletin_date"].count()

In [ ]:
def load_nvd(filename):
  data = dict()
  with open(filename, "r") as f:
    payload = json.load(f)
    for item in payload["CVE_Items"]:
      id = item["cve"]["CVE_data_meta"]["ID"]
      data[id] = item
  return data

In [ ]:
nvd_files = [ f for f in listdir() if path.isfile(f) and f.startswith("nvdcve") and f.endswith(".json")]

nvds = dict()
cve_meta = load_cvrf("allitems-cvrf.xml")

for filename in nvd_files:
  print(f"loading {filename}")
  nvds = {**nvds, **load_nvd(filename)}

In [ ]:
to_remove = []

for cve in cves:
  id = cve["cve"]
  try:
    mod = get_modified(cve_meta, id)
    pub = get_published(cve_meta, id)
    cve["cve_modified"] = mod
    cve["cve_published"] = pub
    if id in nvds and nvds[id]["impact"]:
      if "baseMetricV2" in nvds[id]["impact"]:
        impact = nvds[id]["impact"]["baseMetricV2"]
        score = impact["cvssV2"]["baseScore"]
        severity = impact["severity"]
        cve["nvd_severity"] = severity
        cve["nvd_score"] = score
      if "baseMetricV3" in nvds[id]["impact"]:
        impact = nvds[id]["impact"]["baseMetricV3"]
        cve["nvd_score_v3"] = impact["cvssV3"]["baseScore"]
        cve["nvd_severity_v3"] = impact["cvssV3"]["baseSeverity"]
    else:
      cve["nvd_severity"] = None
      cve["nvd_score"] = None
      print(f"skipping NVD for {id}...")
  except Exception as e:
    print(e)
    print(f"error with cve {id}, skipping")
    to_remove.append(cve)

for cve in to_remove:
  cves.remove(cve)

In [ ]:
output = "drive/My Drive/cves_temp/cves-severity-google-play.json"

with open(output, "w") as f:
  json.dump(cves, f, default=str)

In [ ]:
import csv

def to_csv(cves, out):
  """
  Converts the raw cves to a csv file. The various commit dates are aggregated into
  `commit_start` and `commit_end`, which are respectively the earliest and the
  latest dates that appear in the various commits.
  Duplicated CVEs are left as is, and should be handled in another preprocessing 
  step
  """
  with open(out, "w") as csvfile:
    w = csv.writer(csvfile)
    w.writerow(["id", "bulletin_date", "patch_level", "category", "published", "modified", "commit_start", "commit_end", "nvd_severity", "nvd_score", "nvd_severity_v3", "nvd_score_v3",])
    for cve in cves:
      dates = [d for r in cve["refs"] for d in [r["committed"], r["created"]] if r["committed"] is not None and r["created"] is not None]
      if dates:
        start = min(dates)
        end = max(dates)
      else:
        start = None
        end = None
      try:
        w.writerow([cve["cve"], cve["bulletin_date"], cve["patch_level"], cve.get("category"), cve.get("cve_published"), cve.get("cve_modified"), start, end, cve.get("nvd_severity"), cve.get("nvd_score"), cve.get("nvd_severity_v3"), cve.get("nvd_score_v3")])
      except Exception as e:
        print(f"error {e} with cve {cve['cve']}, skipping...")


In [ ]:
output = "drive/My Drive/cves_temp/aggregated-severity-cat-google-play.csv"

to_csv(cves, output)

In [ ]:
from datetime import datetime

output = "drive/My Drive/cves_temp/bulletin_dates.csv"

bulletin_dates = [(path.basename(url), date.strftime("%Y-%m-%d")) for url, date in urls]

with open(output, "w") as csvfile:
  w = csv.writer(csvfile)
  w.writerow(["security_patch", "date"])
  for row in bulletin_dates:
    w.writerow(row)
